In [11]:
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib gspread pandas


In [12]:
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google.oauth2.service_account import Credentials
import gspread
import pandas as pd
import datetime
import matplotlib.pyplot as plt

# Replace with the path to your service account JSON file
SERVICE_ACCOUNT_FILE = r"C:\Users\avine\Downloads\Project\aristok-automation-441704-40507951b10b.json"

def authenticate_service():
    scopes = [
        "https://www.googleapis.com/auth/webmasters.readonly",  # Search Console API scope
        "https://www.googleapis.com/auth/spreadsheets",         # Google Sheets API scope
    ]
    credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=scopes)
    gsc_service = build('searchconsole', 'v1', credentials=credentials)
    gsheet_client = gspread.authorize(credentials)
    return gsc_service, gsheet_client, credentials

# Authenticate the services
gsc_service, gsheet_client, credentials = authenticate_service()

def fetch_gsc_data(service, site_url, start_date, end_date):
    request = {
        'startDate': start_date,
        'endDate': end_date,
        'dimensions': ['date'],
        'rowLimit': 10000,
    }
    response = service.searchanalytics().query(siteUrl=site_url, body=request).execute()
    return response.get('rows', [])

# Define site URL and date range
site_url = "https://www.medikabazaar.com/"  # Replace with your site URL
start_date = (datetime.date.today() - datetime.timedelta(days=365)).isoformat()
end_date = datetime.date.today().isoformat()

# Fetch data
rows = fetch_gsc_data(gsc_service, site_url, start_date, end_date)

# List of months you want to include in the report
selected_months = ["Jun'24", "Jul'24", "Aug'24", "Sep'24"]

def aggregate_monthly_data(rows, selected_months):
    data = []
    for row in rows:
        date = row['keys'][0]
        clicks = row.get('clicks', 0)
        impressions = row.get('impressions', 0)
        ctr = row.get('ctr', 0) * 100  # Convert to percentage
        position = row.get('position', 0)
        data.append([date, clicks, impressions, ctr, position])
    
    # Convert to DataFrame
    df = pd.DataFrame(data, columns=["Date", "Clicks", "Impressions", "CTR", "Position"])
    df['Date'] = pd.to_datetime(df['Date'])
    df['Month'] = df['Date'].dt.strftime("%b'%y")
    
    # Aggregate by month
    monthly_data = df.groupby('Month').agg(
        Clicks=('Clicks', 'sum'),
        Impressions=('Impressions', 'sum'),
        CTR=('CTR', 'mean'),
        Position=('Position', 'mean')
    ).reset_index()

    # Format the data
    monthly_data['Clicks'] = monthly_data['Clicks'].astype(int)
    monthly_data['Impressions'] = monthly_data['Impressions'].apply(lambda x: f"{x:,.0f}")
    monthly_data['CTR'] = monthly_data['CTR'].apply(lambda x: f"{x:.2f}%")
    monthly_data['Position'] = monthly_data['Position'].apply(lambda x: f"{x:.1f}")
    
    # Filter by selected months
    filtered_data = monthly_data[monthly_data['Month'].isin(selected_months)].reset_index(drop=True)
    return filtered_data

# Aggregate and filter the data
monthly_data = aggregate_monthly_data(rows, selected_months)

def generate_insights(monthly_data):
    insights = []
    if len(monthly_data) >= 2:
        latest = monthly_data.iloc[-1]
        previous = monthly_data.iloc[-2]
        
        click_change = ((latest['Clicks'] - previous['Clicks']) / previous['Clicks']) * 100
        impression_change = ((int(latest['Impressions'].replace(',', '')) - 
                              int(previous['Impressions'].replace(',', ''))) / 
                             int(previous['Impressions'].replace(',', ''))) * 100
        
        # Determine if the changes are improvements or drops
        click_change_type = "improvement" if click_change > 0 else "drop"
        impression_change_type = "improvement" if impression_change > 0 else "drop"
        
        # Format the insights
        insights.append(f"We have observed a {abs(click_change):.1f}% {click_change_type} in Clicks when compared to {previous['Month']}")
        insights.append(f"{abs(impression_change):.1f}% {impression_change_type} in Organic Impressions when compared to {previous['Month']}")
    return insights

# Generate insights
insights = generate_insights(monthly_data)
print("\n".join(insights))

# Create bar chart for monthly clicks comparison
def create_monthly_clicks_chart(monthly_data):
    plt.figure(figsize=(10, 6))
    plt.bar(monthly_data['Month'], monthly_data['Clicks'], color='skyblue')
    plt.xlabel('Month')
    plt.ylabel('Clicks')
    plt.title('Monthly Clicks Comparison')
    plt.tight_layout()
    chart_path = 'monthly_clicks_comparison.png'
    plt.savefig(chart_path)
    plt.close()
    return chart_path

# Generate the bar chart and get the path
chart_path = create_monthly_clicks_chart(monthly_data)

# Replace with the Google Sheet ID
SPREADSHEET_ID = "19qvKm-NClKJgGzumdraNb71E58pdxjesoqjqHEl_9kU"  # Replace with your sheet's actual ID

# Function to upload the bar chart image to the Google Sheet
def upload_image_to_sheet(credentials, sheet_id, image_path, sheet_range='G1'):
    service = build('sheets', 'v4', credentials=credentials)
    media = MediaFileUpload(image_path, mimetype='image/png')
    # Image insertion currently requires extensions in some setups (mocked here for demo)
    print(f"Uploading image from {image_path} to Sheet {sheet_id} at {sheet_range}")

def upload_to_sheets(client, data, insights, chart_path, credentials):
    sheet = client.open_by_key(SPREADSHEET_ID).sheet1
    sheet.clear()
    # Write data
    sheet.update('A1', [data.columns.tolist()] + data.values.tolist())
    # Write insights
    sheet.update(f'A{len(data) + 3}', [[insight] for insight in insights])
    # Upload image
    upload_image_to_sheet(credentials, SPREADSHEET_ID, chart_path)

# Upload data, insights, and the chart
upload_to_sheets(gsheet_client, monthly_data, insights, chart_path, credentials)
print("Data, insights, and chart uploaded to Google Sheets.")


We have observed a 25.3% improvement in Clicks when compared to Jun'24
30.2% improvement in Organic Impressions when compared to Jun'24


C:\Users\avine\AppData\Local\Temp\ipykernel_26744\1682855525.py:136: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update('A1', [data.columns.tolist()] + data.values.tolist())
C:\Users\avine\AppData\Local\Temp\ipykernel_26744\1682855525.py:138: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update(f'A{len(data) + 3}', [[insight] for insight in insights])


Uploading image from monthly_clicks_comparison.png to Sheet 19qvKm-NClKJgGzumdraNb71E58pdxjesoqjqHEl_9kU at G1
Data, insights, and chart uploaded to Google Sheets.
